<a href="https://colab.research.google.com/github/NijilChandran/Sustainability/blob/main/Sustainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
upload = files.upload()

In [4]:
!unzip Participants_Data_WSMLC.zip

Archive:  Participants_Data_WSMLC.zip
  inflating: train.csv               
replace __MACOSX/._train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/._train.csv    
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: sample_submission.csv   
  inflating: __MACOSX/._sample_submission.csv  


In [5]:
df_train = pd.read_csv("train.csv")
df_train.head()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009,1,1,0,0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0
1,2009,1,1,0,30,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0
2,2009,1,1,1,0,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0
3,2009,1,1,1,30,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0
4,2009,1,1,2,0,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0


In [6]:
df_test = pd.read_csv("test.csv")
df_test.head()

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,NaN,NaN,NaN
1,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,NaN,NaN,NaN
2,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,NaN,NaN,NaN
3,2019,1,1,1,30,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0,NaN,NaN,NaN
4,2019,1,1,2,0,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0,NaN,NaN,NaN


In [7]:
df_train.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'Clearsky DHI',
       'Clearsky DNI', 'Clearsky GHI', 'Cloud Type', 'Dew Point',
       'Temperature', 'Pressure', 'Relative Humidity', 'Solar Zenith Angle',
       'Precipitable Water', 'Wind Direction', 'Wind Speed', 'Fill Flag'],
      dtype='object')

In [8]:
X_cols=['Year', 'Month', 'Day', 'Hour', 'Minute','Cloud Type', 'Dew Point',
       'Temperature', 'Pressure', 'Relative Humidity', 'Solar Zenith Angle',
       'Precipitable Water', 'Wind Direction', 'Wind Speed', 'Fill Flag']
y_cols = ['Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI']

In [9]:
X_nmpy =  df_train[X_cols].to_numpy()
y_nmpy =  df_train[y_cols].to_numpy()

In [10]:
X_test_nmpy = df_test[X_cols].to_numpy()
y_test_nmpy = df_test[y_cols].to_numpy()

In [22]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X_nmpy, y_nmpy,test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full,test_size=0.2)

In [23]:
print(X_nmpy.shape[0])
print(X_train_full.shape[0])
print(X_test.shape[0])
print(X_train.shape[0])
print(X_valid.shape[0])

175296
140236
35060
112188
28048


In [39]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [40]:
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential([
	 Dense(600, input_dim=n_inputs, activation='relu'),
	 Dense(300, activation='relu'),
	 Dense(150, activation='relu'),
	 Dense(n_outputs)
  ])
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
n_inputs, n_outputs = X_train.shape[1], y_train.shape[1]
model = get_model(n_inputs, n_outputs)
#set checkpoints
chkpoint_cb = ModelCheckpoint("model_submission_9.h5",save_best_only=True)
early_stopping_cb = EarlyStopping(patience=15,restore_best_weights=True)
# fit model
history = model.fit(X_train, y_train, epochs=150,validation_data=(X_valid, y_valid),
                    callbacks=[chkpoint_cb,early_stopping_cb])
# evaluate model on test set
mse_test = model.evaluate(X_test, y_test)

Epoch 1/150
3506/3506 [==============================] - 24s 7ms/step - loss: 3885.2039 - val_loss: 1123.0065
Epoch 2/150
3506/3506 [==============================] - 27s 8ms/step - loss: 1233.8962 - val_loss: 1132.7506
Epoch 3/150
3506/3506 [==============================] - 21s 6ms/step - loss: 1179.6210 - val_loss: 1124.8905
Epoch 4/150
3506/3506 [==============================] - 21s 6ms/step - loss: 1065.7483 - val_loss: 1401.1917
Epoch 5/150
3506/3506 [==============================] - 21s 6ms/step - loss: 1052.5061 - val_loss: 1626.4639
Epoch 6/150
3506/3506 [==============================] - 25s 7ms/step - loss: 983.7267 - val_loss: 943.6414
Epoch 7/150
3506/3506 [==============================] - 24s 7ms/step - loss: 970.6377 - val_loss: 887.9212
Epoch 8/150
3506/3506 [==============================] - 21s 6ms/step - loss: 927.0159 - val_loss: 829.6967
Epoch 9/150
3506/3506 [==============================] - 22s 6ms/step - loss: 889.3674 - val_loss: 805.8240
Epoch 10/150
3506/

In [28]:
mse_test

622.1611328125

In [29]:
#save model
model.save("model_submission_9.h5")

In [30]:
yhat = model.predict(X_test_nmpy)

In [31]:
df_submission = pd.DataFrame(yhat, columns = y_cols)

In [32]:
df_submission.head()

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,-0.256608,0.167621,-1.134570
1,-1.598443,-0.152085,-2.806393
2,0.071571,-0.070461,0.015488
3,0.083924,-0.059860,0.056938
4,0.085485,-0.058521,0.062178


In [33]:
# convert negative values to 0
df_submission[df_submission < 0] = 0

In [35]:
df_submission.to_csv('my_submission_9.csv', index=False)

In [ ]:
df_submission.shape

(17520, 3)

In [36]:
files.download('my_submission_9.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
files.download("model_submission_9.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>